In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import time
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import argparse
from pmlb import fetch_data
from sklearn import datasets

In [9]:
test = True
# test = False
n_splits = 5

In [8]:
# データセットの読み込み
# fetch data のリスト
reviewed_datasets = [
     'molecular_biology_promoters',
     'car',
     'connect_4',
     'dna',
     '542_pollution',
     '560_bodyfat',
     '1089_USCrime',
     '529_pollen',
     'chess',
     'penguins',
     'bupa',
     'movement_libras',
     'adult',
     'waveform_21',
     'waveform_40',
     'saheart',
     'wine_quality_white',
     'wine_quality_red',
     'irish',
     'mushroom'
 ]

n_splits = 5  # kfoldでcross validationする回数

if test:
    data_kind_list = ["B","M","R"]
    use_data_name = [reviewed_datasets[0], reviewed_datasets[1], reviewed_datasets[4]]
    dfsets = list(map(fetch_data, use_data_name))
else:
    sk_datasets_name = ["load_boston", "load_digits", "load_wine", "fetch_california_housing"]
    sk_datasets = []
    for name in sk_datasets_name:
        print(name)
        data = eval("datasets.{}()".format(name))
        X = pd.DataFrame(data.data, columns=data.feature_names)
        y = pd.DataFrame(data.target,columns=["target"])
        df = pd.concat([X,y], axis=1)
        sk_datasets.append(df)
    use_data_name = reviewed_datasets
    dfsets = list(map(fetch_data, use_data_name)) + sk_datasets
    use_data_name = use_data_name + sk_datasets_name
    # "B": binary classifier, "C": multi classifier, "R":regression
    data_kind_list = ["B", "M", "B", "B", "R","R","R","R", "B", "B", "B","M","B","M","M","B","M","M","B","B","R","R","R","R"]


In [16]:
adult = fetch_data("adult")

In [22]:
kf = KFold(n_splits=n_splits, shuffle=True)
a,b,c,d,e = kf.split(adult)
train_row, test_row = a

In [31]:
train = adult.iloc[train_row, :]
test = adult.iloc[test_row, :]

X_train = train.iloc[:,:-1]  # target以外の列
y_train  = train.iloc[:,-1]  # targetの行
X_test = test.iloc[:,:-1]
y_test = test.iloc[:,-1]

In [40]:
train_left_columns = ["age", "sex", "workclass", "fnlwgt", "education"]
train_right_columns = ["age", "sex"] + list( set(X_train.columns) - set(train_left_columns) )

train_left = X_train.loc[:, train_left_columns]
train_right = X_train.loc[:, train_right_columns]

In [41]:
pipe_lr = Pipeline([("scl",StandardScaler()),("est",LogisticRegression()) ])
pipe_gb = Pipeline([("scl",StandardScaler()),("est",GradientBoostingClassifier()) ])

In [45]:
pipe_lr.fit(X_train, y_train)
pipe_gb.fit(X_train, y_train)

Pipeline(steps=[('scl', StandardScaler()),
                ('est', GradientBoostingClassifier())])

In [46]:
pipe_lr.predict(X_test)
pipe_gb.predict(X_test)

array([1, 0, 1, ..., 1, 1, 0])